In [1]:
# 12/5(금) 9:20

# Encoder–Decoder 구조

- Encoder–Decoder 구조는 어떤 형태의 입력 시퀀스를 받아 **의미를 해석**한 뒤, 새로운 **출력 시퀀스를 생성**해야 하는 거의 모든 AI 문제를 해결하는 딥러닝 모델 구조다.
- 이 구조는 Encoder와 Decoder 두개의 딥러닝 모델을 연결한 구조로 **입력 데이터를 하나의 표현으로 압축한 뒤, 이를 다시 출력 데이터로 변환하는 방식**으로 동작한다.

- **Encoder Network**
  - 입력 데이터를 해석(이해)하는 역할을 수행.
  - 입력 시퀀스에 담긴 의미적 정보를 하나의 고정된 벡터 형태로 요약.

- **Decoder Network**
  - Encoder가 생성한 요약 정보를 바탕으로 최종 출력을 생성.
  - 즉, Encoder의 “이해 결과”를 이용해 새로운 시퀀스를 만들어낸다.

## Seq2Seq (Sequence-to-Sequence)

Seq2Seq 모델: **Encoder–Decoder 구조를 RNN(Recurrent Neural Network) 계열에 적용한 대표적인 시퀀스 변환 모델**.  
입력과 출력이 모두 “시퀀스(sequence)” 형태라는 점에서 *Sequence-to-Sequence*라는 이름이 붙었다.

### Encoder의 역할: 입력 시퀀스 이해 및 Context Vector 생성

Encoder는 입력으로 들어온 **전체 시퀀스**(sequence)를 순차적으로 처리한 뒤,  그 의미를 **하나의 고정 길이 벡터**(Vector)로 압축하여 출력한다.  
이 벡터를 **Context Vector**(컨텍스트 벡터)라고 한다.
- **Context Vector란?**  
  - 입력 시퀀스 전체의 의미, 문맥, 핵심 정보를 요약해 담고 있는 벡터 표현이다.
  - **기계 번역**(Machine Translation)의 경우  
    - 번역할 원문 문장에서 **번역 결과를 생성하는 데 필요한 핵심 의미 정보**(feature)
  - **챗봇**(Chatbot)의 경우  
    - 사용자가 입력한 질문에서 **적절한 답변을 생성하는 데 필요한 의미 정보**(feature)

### Decoder의 역할: Context Vector를 바탕으로 출력 시퀀스 생성

Decoder는 Encoder가 출력한 **Context Vector를 입력으로 받아**, 이를 바탕으로 **목표 출력 시퀀스**를 한 토큰(token)씩 순차적으로 생성.

- **기계 번역**(Machine Translation)의 경우  
  - 입력 문장의 의미를 반영한 **번역 문장** 생성.
- **챗봇**(Chatbot)  
  - 질문에 대한 **자연스러운 답변 문장** 생성.

Decoder는 매 시점(time step)마다
  - 이전에 생성한 단어
  - 그리고 Context Vector에 담긴 입력 문맥
을 함께 고려해 다음 단어를 예측.


![seq2seq](figures/seq2seq.png)

# Seq2Seq 를 이용한 Chatbot 모델 구현
- Encoder를 이용해 질문의 특성을 추출하고 Decoder를 이용해 답변을 생성한다.

# Chatbot Dataset

- https://github.com/songys/Chatbot_data
- columns
    - Q: 질문
    - A: 답
    - label: 일상다반사 0, 이별(부정) 1, 사랑(긍정) 2
- **Download**

![dataset](figures/chatbot.png)

# Chatbot Dataset Loading 및 확인

## 데이터셋 다운로드 및 확인

In [2]:
import requests
import os

url = "https://raw.githubusercontent.com/songys/Chatbot_data/refs/heads/master/ChatbotData.csv"
res = requests.get(url)
if res.status_code == 200:
    with open("data/chatbot_data.csv", "wt", encoding="utf-8") as fw:
        fw.write(res.text)
else:
    print("다운 실패:", res.status_code)

In [3]:
import pandas as pd

df = pd.read_csv("data/chatbot_data.csv", encoding="utf-8")
df.shape

(11823, 3)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11823 non-null  object
 1   A       11823 non-null  object
 2   label   11823 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB


In [5]:
df.drop(columns='label', inplace=True)

In [6]:
df.head(10)

,Q,A
0,12시 땡!,하루가 또 가네요.
1,1지망 학교 떨어졌어,위로해 드립니다.
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
4,PPL 심하네,눈살이 찌푸려지죠.
5,SD카드 망가졌어,다시 새로 사는 게 마음 편해요.
6,SD카드 안돼,다시 새로 사는 게 마음 편해요.
7,SNS 맞팔 왜 안하지ㅠㅠ,잘 모르고 있을 수도 있어요.
8,SNS 시간낭비인 거 아는데 매일 하는 중,시간을 정하고 해보세요.
9,SNS 시간낭비인데 자꾸 보게됨,시간을 정하고 해보세요.


In [7]:
df.tail(10)

,Q,A
11813,회사에 좋아하는 남자가 생겼어 어떡하지?,사랑하기 힘든 관계인가봐요.
11814,회사에서 어떤 사람이랑 자꾸 눈 마추쳐.,눈 마주치는 게 우연인지 잘 살펴 보세요.
11815,회식 중이라고 하는데 연락이 안돼.,정신 없이 바쁠지도 몰라요. 조금만 더 기다려보고 물어보는게 좋을 것 같아요.
11816,회식하는데 나만 챙겨줘. 썸임?,호감이 있을 수도 있어요. 그렇지만 조금 더 상황을 지켜보세요.
11817,후회 없이 사랑하고 싶어,진심으로 다가가 보세요.
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.
11820,흑기사 해주는 짝남.,설렜겠어요.
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.
11822,힘들어서 결혼할까봐,도피성 결혼은 하지 않길 바라요.


In [8]:
df.isnull().sum()

Q    0
A    0
dtype: int64

# Dataset, DataLoader 정의

## Tokenization

### Subword방식

In [9]:
# 토큰화를 위해서 문장을 q + a 형식으로 만든다. 
# 어휘사전을 만들 때 Q와 A에 있는 모든 단어들이 다 들어가게 하기 위해.
question_texts = df['Q']
answer_texts = df['A']

all_texts = list(question_texts+" "+answer_texts)  # series + 문자열 + series (원소 단위 연산)

In [10]:
all_texts[:10]

['12시 땡! 하루가 또 가네요.',
 '1지망 학교 떨어졌어 위로해 드립니다.',
 '3박4일 놀러가고 싶다 여행은 언제나 좋죠.',
 '3박4일 정도 놀러가고 싶다 여행은 언제나 좋죠.',
 'PPL 심하네 눈살이 찌푸려지죠.',
 'SD카드 망가졌어 다시 새로 사는 게 마음 편해요.',
 'SD카드 안돼 다시 새로 사는 게 마음 편해요.',
 'SNS 맞팔 왜 안하지ㅠㅠ 잘 모르고 있을 수도 있어요.',
 'SNS 시간낭비인 거 아는데 매일 하는 중 시간을 정하고 해보세요.',
 'SNS 시간낭비인데 자꾸 보게됨 시간을 정하고 해보세요.']

In [11]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

tokenizer = Tokenizer(
    BPE(unk_token="<unk>")
)
tokenizer.pre_tokenizer = Whitespace()
trainer = BpeTrainer(
    vocab_size=10_000,    # 최대 어휘 수
    min_frequency=5,      # 어휘사전에 등록할 단어의 최소 빈도 수 (5회 이상은 나와야 등록)
    continuing_subword_prefix='##',    # 연결 subword 앞에 붙일 접두어를 ##로 지정. cowork: co + ##work
    special_tokens=["<pad>", "<unk>", "<sos>"]   # <sos>는 문장의 시작을 의미하는 특수 토큰
)
tokenizer.train_from_iterator(all_texts, trainer=trainer)

In [12]:
print("총 어휘 수:", tokenizer.get_vocab_size())

총 어휘 수: 7041


In [13]:
encode = tokenizer.encode("오늘 날씨가 너무 좋습니다. 이런 날씨에 뭘 하면 좋을까요?")

In [14]:
encode.tokens

['오늘', '날씨가', '너무', '좋습니다', '.', '이런', '날씨', '##에', '뭘', '하면', '좋을까요', '?']

In [15]:
encode.ids

[2290, 3851, 2258, 5914, 8, 2752, 2841, 1278, 527, 2530, 5532, 20]

In [17]:
tokenizer.id_to_token(1500)  # id로 토큰 문자열 조회
tokenizer.token_to_id("하면") # 토큰 문자열로 id (정수)를 조회

2530

### Tokenizer 저장

In [18]:
tokenizer.save("saved_models/chatbot_bpe.json")

In [19]:
load_tokenizer = Tokenizer.from_file("saved_models/chatbot_bpe.json")

## Dataset, DataLoader 정의


### Dataset 정의 및 생성
- 모든 문장의 토큰 수는 동일하게 맞춰준다.
    - DataLoader는 batch 를 구성할 때 batch에 포함되는 데이터들의 shape이 같아야 한다. 그래야 하나로 묶을 수 있다.
    - 문장의 최대 길이를 정해주고 **최대 길이보다 짧은 문장은 `<PAD>` 토큰을 추가**하고 **최대길이보다 긴 문장은 최대 길이에 맞춰 짤라준다.**

In [20]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "mps"  # 맥 M1 이상 쓰는 사람들 

In [22]:
class ChatbotDataset(Dataset):
    
    def __init__(self, question_texts, answer_texts, max_length, tokenizer):
        """
        Args:
            question_texts (list[str]):질문 text 리스트 ["질문1", "질문2, ...]
            answer_texts (list[str]): 답변 text 리스트 ["답변1", "답변2", ...]
            max_length (int): 개별 문장의 최대 토큰 수
            tokenizer (Tokenizer)
        """
        self.max_length = max_length
        self.tokenizer = tokenizer
        # "질문" -> Tensor (토큰 ID)
        self.question_texts = [self.__process_sequence(q) for q in question_texts]
        self.answer_texts = [self.__process_sequence(a) for a in answer_texts]

    def __pad_token_sequence(self, token_sequence):
        """ 
        token_sequence를 self.max_length 길이에 맞추는 메소드.
        max_length보다 적으면 <pad> 추가, 크면 잘라낸다. 
        Args: 
            token_sequence (list[str]): 한 문장의 토큰 ID 리스트 [2234, 7100, 257, ...]
        Returns:
            list[int]: 길이를 max_length에 맞춘 토큰화 한 토큰 ID 리스트
        """
        pad_token = self.tokenizer.token_to_id('<pad>')
        seq_length = len(token_sequence)
        if seq_length > self.max_length:   # 잘라내기
            result = token_sequence[:self.max_length]
        else: # <pad> 추가 (padding 처리)
            result = token_sequence + [pad_token] * (self.max_length - seq_length)

        return result

    def __process_sequence(self, text):
        """
        한 문장 (text-str)을 받아서 token화 한 뒤 max_length에 개수를 맞춰서 반환. 
        max_length에 맞추는 작업은 __pad_token_sequence()를 이용
        Args:
            text (str): 토큰화 할 문장
        Returns:
            torch.Tensor[int64]: 토큰화 한 토큰 ID 리스트
        """
        encode = self.tokenizer.encode(text)
        token_ids = encode.ids    # "나는 학생이다." -> [4020, 1003, 3932]
        # [4020, 1003, 3932] -> [4020, 1003, 3932, 0, 0, 0] 패딩 처리.
        return torch.tensor(self.__pad_token_sequence(token_ids), dtype=torch.int64)

    def __len__(self):
        return len(self.question_texts)

    def __getitem__(self, index):
        """ 
        index의 (question, answer) 쌍을 반환.
        Args: 
            index(int): 몇 번 질문 - 답변 쌍인지 index
        Return:
            tuple[Tensor(int64), Tensor(int64)]
        """
        q = self.question_texts[index]
        a = self.answer_texts[index]
        return q, a


In [30]:
# max_length. 가장 긴 문장의 토큰 수 
[sent for sent in question_texts]

['12시 땡!',
 '1지망 학교 떨어졌어',
 '3박4일 놀러가고 싶다',
 '3박4일 정도 놀러가고 싶다',
 'PPL 심하네',
 'SD카드 망가졌어',
 'SD카드 안돼',
 'SNS 맞팔 왜 안하지ㅠㅠ',
 'SNS 시간낭비인 거 아는데 매일 하는 중',
 'SNS 시간낭비인데 자꾸 보게됨',
 'SNS보면 나만 빼고 다 행복해보여',
 '가끔 궁금해',
 '가끔 뭐하는지 궁금해',
 '가끔은 혼자인게 좋다',
 '가난한 자의 설움',
 '가만 있어도 땀난다',
 '가상화폐 쫄딱 망함',
 '가스불 켜고 나갔어',
 '가스불 켜놓고 나온거 같아',
 '가스비 너무 많이 나왔다.',
 '가스비 비싼데 감기 걸리겠어',
 '가스비 장난 아님',
 '가장 확실한 건 뭘까?',
 '가족 여행 가기로 했어',
 '가족 여행 고고',
 '가족 여행 어디로 가지?',
 '가족 있어?',
 '가족관계 알려 줘',
 '가족끼리 여행간다.',
 '가족들 보고 싶어',
 '가족들이랑 서먹해',
 '가족들이랑 서먹해졌어',
 '가족들이랑 어디 가지?',
 '가족들이랑 여행 갈거야',
 '가족여행 가야지',
 '가족이 누구야?',
 '가족이랑 여행 가려고',
 '가족한테 스트레스 풀었어',
 '가출할까?',
 '가출해도 갈 데가 없어',
 '간만에 떨리니까 좋더라',
 '간만에 쇼핑 중',
 '간만에 휴식 중',
 '간식 뭐 먹을까',
 '간식 추천',
 '간장치킨 시켜야지',
 '간접흡연 싫어',
 '갈까 말까 고민 돼',
 '갈까 말까?',
 '감 말랭이 먹고 싶다.',
 '감 말랭이 먹어야지',
 '감기 같애',
 '감기 걸린 것 같아',
 '감기 기운이 있어',
 '감기 들 거 같애',
 '감기가 오려나',
 '감기약이 없어',
 '감기인거 같애',
 '감미로운 목소리 좋아',
 '감정이 쓰레기통처럼 엉망진창이야',
 '감정컨트롤을 못하겠어',
 '감정컨트롤이 안돼',
 '감히 나를 무시하는 애가 있어',
 '갑자기 나쁜 생각이 막 들더라',
 '갑자기 눈물 나',

In [23]:
dataset = ChatbotDataset(
    list(question_texts),
    list(answer_texts), 
    5, 
    tokenizer
)

In [24]:
len(dataset)

11823

In [25]:
dataset[0]

(tensor([  10, 1903, 1329,  368,    3]),
 tensor([6119,  378,   47, 2252,    8]))

### Trainset / Testset 나누기
train : test = 0.95 : 0.05

### DataLoader 생성

# 모델 정의

## Seq2Seq 모델 정의
- Seq2Seq 모델은 Encoder와 Decoder의 입력 Sequence의 길이와 순서가 자유롭기 때문에 챗봇이나 번역에 이상적인 구조다.
    - 단일 RNN은 각 timestep 마다 입력과 출력이 있기 때문에 입/출력 sequence의 개수가 같아야 한다.
    - 챗봇의 질문/답변이나 번역의 대상/결과 문장의 경우는 사용하는 어절 수가 다른 경우가 많기 때문에 단일 RNN 모델은 좋은 성능을 내기 어렵다.
    - Seq2Seq는 **입력처리(질문,번역대상)처리 RNN과 출력 처리(답변, 번역결과) RNN 이 각각 만들고 그 둘을 연결한 형태로 길이가 다르더라도 상관없다.**

## Encoder
Encoder는 하나의 Vector를 생성하며 그 Vector는 **입력 문장의 의미**를 N 차원 공간 저장하고 있다. 이 Vector를 **Context Vector** 라고 한다.    
![encoder](figures/seq2seq_encoder.png)

## Decoder
- Encoder의 출력(context vector)를 받아서 번역 결과 sequence를 출력한다.
- Decoder는 매 time step의 입력으로 **이전 time step에서 예상한 단어와 hidden state값이** 입력된다.
- Decoder의 처리결과 hidden state를 Estimator(Linear+Softmax)로 입력하여 **입력 단어에 대한 번역 단어가 출력된다.** (이 출력단어가 다음 step의 입력이 된다.)
    - Decoder의 첫 time step 입력은 문장의 시작을 의미하는 <SOS>(start of string) 토큰이고 hidden state는 context vector(encoder 마지막 hidden state) 이다.

![decoder](figures/seq2seq_decoder.png)

## Seq2Seq 모델

- Encoder - Decoder 를 Layer로 가지며 Encoder로 질문의 feature를 추출하고 Decoder로 답변을 생성한다.

### Teacher Forcing
- **Teacher forcing** 기법은, RNN계열 모델이 다음 단어를 예측할 때, 이전 timestep에서 예측된 단어를 입력으로 사용하는 대신 **실제 정답 단어(ground truth) 단어를** 입력으로 사용하는 방법이다.
    - 모델은 이전 시점의 출력 단어를 다음 시점의 입력으로 사용한다. 그러나 모델이 학습할 때 초반에는 정답과 많이 다른 단어가 생성되어 엉뚱한 입력이 들어가 학습이 빠르게 되지 않는 문제가 있다.
- **장점**
    - **수렴 속도 증가**: 정답 단어를 사용하기 때문에 모델이 더 빨리 학습할 수있다.
    - **안정적인 학습**: 초기 학습 단계에서 모델의 예측이 불안정할 때, 잘못된 예측으로 인한 오류가 다음 단계로 전파되는 것을 막아줍니다.
- **단점**
    - **노출 편향(Exposure Bias) 문제:** 실제 예측 시에는 정답을 제공할 수 없으므로 모델은 전단계의 출력값을 기반으로 예측해 나가야 한다. 학습 과정과 추론과정의 이러한 차이 때문에 모델의 성능이 떨어질 수있다.
        - 이런 문제를 해결하기 학습 할 때 **Teacher forcing을 random하게 적용하여 학습시킨다.**
![seq2seq](figures/seq2seq.png)

# 학습

## 모델생성

## loss함수, optimizer

## train/evaluation 함수 정의

### train 함수정의

### Test 함수

### Training

# 결과확인

- Sampler:
    -  DataLoader가 Datatset의 값들을 읽어서 batch를 만들때 index 순서를 정해주는 객체.
    -  DataLoader의 기본 sampler는 SequentialSampler 이다. shuffle=True 일경우 RandomSampler: 랜덤한 순서로 제공.

# 학습모델을 이용한 대화